In [1]:
import neurite_sandbox as nes
import numpy as np

ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.15; Detected an installation of version 2.12.0. Please upgrade TensorFlow to proceed.

In [ ]:
def perlin_generation(
    num_to_gen,
    num_labels_range=[5,20],
    shapes_im_max_std_range=[0.5, 5],
    shapes_warp_max_std_range=[4.0, 15.0],
    std_min_range=[0.01, 0.1],
    std_max_range=[0.2, 1],
    lab_int_interimage_std_range=[0.01, 0.1],
    warp_std_range=[1, 20],
    bias_res_range=[32, 50],
    bias_std_range=[0.1, 1.0],
    blur_std_range=[0.5, 5],
    min_density=0.001,
):

    # Gen parameters
    num_labels = np.random.randint(low=num_labels_range[0], high=num_labels_range[1])
    shapes_im_max_std = np.random.uniform(shapes_im_max_std_range[0], shapes_im_max_std_range[1])
    shapes_warp_max_std = np.random.uniform(shapes_warp_max_std_range[0], shapes_warp_max_std_range[1])
    std_min = np.random.uniform(std_min_range[0], std_min_range[1])
    std_max = np.random.uniform(std_max_range[0], std_max_range[1])
    lab_int_interimage_std = np.random.uniform(lab_int_interimage_std_range[0], lab_int_interimage_std_range[1])
    warp_std = np.random.uniform(warp_std_range[0], warp_std_range[1])
    bias_res = np.random.uniform(bias_res_range[0], bias_res_range[1])
    bias_std = np.random.uniform(bias_std_range[0], bias_std_range[1])
    blur_std = np.random.uniform(blur_std_range[0], blur_std_range[1])

    # Gen tasks
    images, label_maps, lab = nes.tf.utils.synth.perlin_nshot_task(in_shape=(256,256),
                                                                  num_gen=num_to_gen,
                                                                  num_label=num_labels,
                                                                  shapes_im_scales=(32, 64, 128),
                                                                  shapes_warp_scales=(16, 32, 64, 128),
                                                                  shapes_im_max_std=shapes_im_max_std,
                                                                  shapes_warp_max_std=shapes_warp_max_std,
                                                                  min_int=0,
                                                                  max_int=1,
                                                                  std_min=std_min,
                                                                  std_max=std_max,
                                                                  lab_int_interimage_std=lab_int_interimage_std,
                                                                  warp_std=warp_std,
                                                                  warp_res=(8, 16, 32, 64),
                                                                  bias_res=bias_res,
                                                                  bias_std=bias_std,
                                                                  blur_std=blur_std)


    all_labels = np.unique(lab).tolist()

    background_labels = []
    for lab in all_labels:
        for lm in label_maps:
            if np.mean(lm[..., lab]) < min_density:
                all_labels.remove(lab)
                background_labels.append(background_labels)
                break

    assert len(all_labels) > 0

    label_map_big_labels = [f[..., np.array(all_labels)] for f in label_maps]

    label_map_big_labels_wbg = []
    foreground_labels = []
    background_labels = []
    for f in label_map_big_labels:
        foreground_label = (np.sum(f, axis=-1))[..., np.newaxis]
        background_label = (1-np.sum(f, axis=-1))[..., np.newaxis]
        foreground_labels.append(foreground_label)
        background_labels.append(background_label)
        new_f = np.concatenate([background_label, f], -1)
        label_map_big_labels_wbg.append(new_f)
    flattened_label_map_big_labels_wbg = [np.argmax(f, -1) for f in label_map_big_labels_wbg]

    return images, flattened_label_map_big_labels_wbg

In [ ]:
images, label_maps = perlin_generation(num_to_gen=10, min_density=0.003)